In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

In [2]:
f = open(r'images/templeSparseRing/templeSR_par.txt','r')
assert f is not None
n = int(f.readline())

#First Image
l = f.readline().split()
im1_fn = l[0]
k1 = np.array([float(i) for i in l[1:10]]).reshape((3,3))
r1 = np.array([float(i) for i in l[10:19]]).reshape((3,3))
t1 = np.array([float(i) for i in l[19:22]]).reshape((3,1))

#Second Image
l = f.readline().split()
im2_fn = l[0]
k2 = np.array([float(i) for i in l[1:10]]).reshape((3,3))
r2 = np.array([float(i) for i in l[10:19]]).reshape((3,3))
t2 = np.array([float(i) for i in l[19:22]]).reshape((3,1))

In [3]:
im1 = cv.imread(r'images/templeSparseRing/'+im1_fn,cv.IMREAD_COLOR)
im2 = cv.imread(r'images/templeSparseRing/'+im2_fn,cv.IMREAD_COLOR)
assert im1 is not None
assert im2 is not None

In [13]:
sift = cv.xfeatures2d.SIFT_create()
kp1,desc1 = sift.detectAndCompute(im1,None)
kp2,desc2 = sift.detectAndCompute(im2,None)

FLANN_INDEX_KDTREE = 1

indx_para = dict(algorithm = FLANN_INDEX_KDTREE,trees = 5)
search_para = dict(checks = 100)
flann = cv.FlannBasedMatcher(indx_para,search_para)
matches = flann.knnMatch(desc1,desc2, k= 2)

good = []
pts1,pts2 = [],[]

for i,(m,n) in enumerate(matches):
    if m.distance < 0.7 *n.distance:
        good.append(m)
        pts1.append(kp1[m.queryIdx].pt)
        pts2.append(kp2[m.trainIdx].pt)

pts1 = np.array(pts1)
pts2 = np.array(pts2)


In [17]:
F,mask = cv.findFundamentalMat(pts1,pts2,cv.FM_RANSAC)
E = k2.T @ F@ k1